In [10]:
import requests 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from rich.console import Console
from rich.markdown import Markdown
console = Console()
import os
from dotenv import load_dotenv

In [6]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website :

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')

        self.title = soup.title.string if soup.title else None
        print(self.title if self.title else "No title found")

        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"



In [10]:
an = Website("https://anthropic.com")
an.links

Home \ Anthropic


['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/claude-code',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/solutions/government',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/econom

In [11]:
link_system_prompt = " you are provided with the number of links that have been found on the webpage"
link_system_prompt+= " you are an assistant the will collect the relevant links that only suitable for company brochure\
such as links related to about page, politics, contact"
link_system_prompt+= "You are an assistant that MUST return ONLY valid JSON. No prose, no comments, no Markdown\
Return exactly this shape:,  as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "MIDDLE EAST", "url": "https://middleeast.full.url/middleeast"}
    ]
}
Rules:
- Output must be a single JSON object parsable by Python json.loads.
- Do not truncate URLs. If a relevant link doesn't exist, omit it (do NOT invent).
- Prefer About / Company / Careers / Blog / Contact / Social profiles relevant to a brochure.
"""
link_system_prompt+="your main focus on the links of politics related to IRAN ISRAEL AND US"

In [12]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt


In [7]:
from rich.console import Console
import requests
from rich.markdown import Markdown
console = Console()
import json 
import requests
import requests
import json
import re
from urllib.parse import urljoin

def get_links(url):
    website = Website(url)

    # Strongly worded system prompt helps; JSON mode enforces syntax.
    response = requests.post(
        "http://localhost:11434/api/chat",
        json={
            "model": "llama3",
            "messages": [
                {"role": "system", "content": link_system_prompt},
                {"role": "user", "content": get_links_user_prompt(website)}
            ],
            # 🔒 Force JSON-only output
            "format": "json",
            "stream": False,
            # Make the model deterministic/less chatty
            "options": {"temperature": 0}
        },
      
    )

    response.raise_for_status()
    data = response.json()

    # In JSON mode, this should already be a clean JSON string
    text = data["message"]["content"].strip()

    # Some UIs wrap JSON in ```json ... ```; strip if present
    m = re.search(r"```(?:json)?\s*(\{.*\})\s*```", text, re.DOTALL)
    if m:
        text = m.group(1)

    try:
        result = json.loads(text)
    except json.JSONDecodeError as e:
        # Helpful debug so you can see exactly what the model returned
        raise ValueError(f"Model did not return valid JSON.\nError: {e}\nRaw text:\n{text}") from e

    # Validate expected shape
    if not isinstance(result, dict) or "links" not in result or not isinstance(result["links"], list):
        raise ValueError(f"Unexpected shape from model: {result}")

    # Normalize/clean items
    cleaned = {"links": []}
    for item in result["links"]:
        if not isinstance(item, dict):
            continue
        link_type = item.get("type")
        link_url = item.get("url")
        if not isinstance(link_type, str) or not isinstance(link_url, str):
            continue
        # Resolve relative URLs to absolute
        abs_url = urljoin(website.url, link_url)
        cleaned["links"].append({"type": link_type.strip(), "url": abs_url})

    return cleaned




In [ ]:
get_links("https://anthropic.com")

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result


In [ ]:
system_prompt = """ you ar an assistant and now you have relevant links as well, whatever the information you get
make the brochure relevant for customers, investors, stakeholders, and people who are interseted, remember response in 
markdown with beatiful heading and bold text so it can attract potential people"""

In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
def messages_are(website):
     return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": get_brochure_user_prompt("ANTHROPIC", "https://anthropic.com")}
    ]

 

In [ ]:
from rich.console import Console
import requests
from rich.markdown import Markdown
console = Console()

def summarize(company_name, url):
    website = Website(url)
    response = requests.post(
        "http://localhost:11434/api/chat",
        json={
            "model": "llama3",
            "messages": messages_are(website),
            "stream": False
        }
    )
    data = response.json()
    return data['message']['content']




In [ ]:
summary = summarize("ANTHROPIC", "https://anthropic.com")

In [ ]:
md = Markdown(summary)
console.print(md)

<h1>URDU BROCHURE</h1>

In [ ]:
urdu_system_prompt = """ you are an assistant and provided with website' brochure, now convert this 
brochure into Urdu language. Remember response in markdown. Create this brochure into 
Spanish language. """

In [ ]:
def urdu_user_prompt(company_name, url):
    user_prompt = f"You are looking at the brochure of {company_name}.\n"
    user_prompt += "Here is the brochure in English. Please translate it into URDU:\n\n"
    user_prompt += summarize(company_name, url)   
    return user_prompt


In [ ]:
def messages(website):
    return [
    {"role": "system", "content": urdu_system_prompt},
    {"role": "user", "content": urdu_system_prompt("ANTHROPIC", "https://anthropic.com")}
]


In [ ]:
from rich.console import Console
import requests
from rich.markdown import Markdown
console = Console()

def urdu(company_name, url):
    website = Website(url)
    response = requests.post(
        "http://localhost:11434/api/chat",
        json={
            "model": "llama3",
            "messages": messages(website),
            "stream": False
        }
    )
    data = response.json()
    return data['message']['content']




In [ ]:
foreign = urdu("ANTHROPIC", "https://anthropic.com")

In [ ]:
md = Markdown(foreign)
console.print(md)